![Exeter Logo](https://github.com/automat9/Business-Analytics/blob/master/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/visuals/Exeter%20Logo.png?raw=true)

<h3 align="center">MSc Business Analytics</h3>
  
<h2 align="center">Analytics and Visualisation for Managers and Consultants</h2>

<h1 align="center">Logistics Insights Dashboard</h1>

<h3 align="center">Student ID: 730068016</h3>
<h3 align="center">Word Count: 3276</h3>


| Description       | Link                                                                 |
|-------------------|----------------------------------------------------------------------|
| Reflective Blog   | [Click Here](https://ele.exeter.ac.uk/mod/oublog/view.php?id=3234123) |
| Dataset Used      | [Click Here](https://www.kaggle.com/datasets/shivaiyer129/supply-chain-data-set) |
| Supporting Files   | [Click Here](https://github.com/automat9/Business-Analytics/tree/master/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants) |

| Table of Contents|
|---------------------|
|1. Introduction|
|2. Context|
|3. Aims and objectives|
|4. Project Dashboard|
|5. Decision Making Process|
|5. Background|
|6. Review of analytics methods applied|
|7. Review of selected tools and potential alternatives|
|8. Review of chosen dataset|
|9. Code Discussion|
|10. Personal Reflection|
|11. Conclusion|
|12. Reference|

# 1. Introduction
This report aims to analyse and explain the development of a ‘Logistics Insight Dashboard’, which shows several simple, yet effective visuals that provide key insights into operational activities of a third-party logistics company (3PL). The report reviews the dataset used, the decision-making process, as well as the analytical techniques used in the development of the dashboard to provide a comprehensive overview of the dashboard creation process. This report will help to highlight how visual aids can assist the managing director at a 3PL company make better informed decisions and effectively identify areas for improvement. 

# 2. Context
Companies working in the logistics sector face numerous challenges, including effective inventory management, carrier coordination, and delivery timeliness. Given the complexity of supply chains and potential risks such as human error or process inefficiencies, it is crucial to continuously monitor the operations of your company, as well as those of clients and suppliers. This proactive approach allows for quick detection of changes or inefficiencies that could impact the entire supply chain. The role of simple and intuitive dashboards in this is pivotal, as they simplify complex processes, allowing decision-makers to spot patterns, or setups that may lead to inefficiencies.  

The managing director at a 3PL company is seeking to enhance his company’s operational efficiency and service quality by implementing data-driven dashboards into his decision-making processes. These dashboards will provide key insights and highlights inefficiencies. The company works with a few carriers and is interested in analysing their performance and reliability. This analysis will help to determine whether changes to their carrier partnerships are necessary to improve service levels. Additionally, the company seeks a broader understanding of its operations. Key areas include identifying which products are most likely to late dispatch, assessing the volume of orders processed through each of the three ports, and exploring whether some orders could be rerouted to distribute the workloads more effectively.

The managing director has tasked one of his logistics analysts to create a dashboard that illustrates all this data, and then to present his findings.


# 3. Aims and objectives
Aim:
To develop a clear and comprehensive dashboard that visualises key insights, highlights areas for improvement, and supports the managing director in making informed decisions.

Objectives:

a)	To study relevant literature on effective dashboard creation, identifying key principles, such as accessibility, simplicity and storytelling. This will ensure that the dashboard is grounded in proven design practices. 

b)	To develop an intuitive and accessible dashboard consisting of five visuals that address critical operational metrics that the managing director is interested in. This will be achieved by gathering, cleaning and organising a relevant dataset.

c)	To review and evaluate the dashboard’s design, code and techniques used, identifying potential areas for improvement. This will ensure the dashboard delivers what it promised to, remains user-friendly, and effectively supports data-driven decision-making.


# 4. Project Dashboard


In [7]:
### Data Preparation ###

# Uncomment to install required libraries
#!pip install dash
#!pip install plotly
#!pip install pandas

# Import required libraries
from dash import Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Read the dataset
data = pd.read_excel("https://raw.githubusercontent.com/automat9/Business-Analytics/4345e44423e83e68e5e92815c37036670c9acad8/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/datasets/data_cleaned.xlsx",sheet_name=None)

# Assign dataframes based on sheet names
df1 = data['orders_carrier_pie']
df2 = data['carrier_late_bar']
df3 = data['top 20_late_bar']
df4 = data['orders_port_bar']
df5 = data['ontime_ahead_gauge'] 

### Data Processing ###

# Fig1: Orders handled by carrier
df1_grouped = df1.groupby('Carrier').size().reset_index(name='Order Count')

# Fig2: Chance of late delivery by carrier
carrier_totals = df2.groupby('Carrier').size().reset_index(name='Total Deliveries')
carrier_late = df2[df2['Ship Late Day count'] > 0].groupby('Carrier').size().reset_index(name='Late Deliveries')
df2_processed = pd.merge(carrier_totals, carrier_late, on='Carrier', how='left')
df2_processed['Late Deliveries'] = df2_processed['Late Deliveries'].fillna(0)
df2_processed['Late Delivery Probability'] = (df2_processed['Late Deliveries'] / df2_processed['Total Deliveries']) * 100

# Fig3: Top 20 products by late dispatch
df3_ordered = (
    df3.query('`Ship Late Day count` > 0')
    ['Product ID']
    .astype(str)
    .value_counts()
    .nlargest(20)
    .reset_index(name='Late Order Count')
    .rename(columns={'index': 'Product ID'}))
df3_ordered['colour'] = df3_ordered['Late Order Count'].apply(
    lambda x: 'lightgreen' if x < 10 else 'khaki' if 10 <= x <= 13 else 'lightcoral')

# Fig4: Orders handled by port
df4_grouped = df4.groupby('Origin Port').size().reset_index(name='Number of Orders')

# Fig5: Ship on-time/ahead of schedule
total_deliveries = len(df5)
df5_ok_deliveries = len(df5[df5['Ship Late Day count'] <= 0])
df5_ok_percentage = (df5_ok_deliveries / total_deliveries) * 100



### Initialise dash ###

app = Dash(__name__)

### Figure Setup ###

fig1 = px.pie(
    df1_grouped,
    values = 'Order Count',
    names = 'Carrier',
    title = 'Orders Handled by Carrier')

fig2 = px.bar(
    df2_processed,
    x='Carrier',
    y='Late Delivery Probability',
    title="Chance of Late Delivery by Carrier",
    labels={'Late Delivery Probability': 'Probability (%)'},
    text='Late Delivery Probability',
    color='Carrier')
fig2.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig2.update_layout(yaxis=dict(range=[0, 3.5]), coloraxis=dict(colorbar=dict(title='Carrier'), colorscale=[[0, 'blue'], [0.5, 'red'], [1, 'green']]))

fig3 = px.bar(
    df3_ordered,
    x='Product ID',
    y='Late Order Count',
    labels={'Late Order Count': 'Late Dispatch'},
    title='Products by Most Late Dispatch',
    text='Late Order Count')
fig3.update_traces(marker=dict(color=df3_ordered['colour']),texttemplate='%{text}', textposition='outside')
fig3.update_layout(yaxis=dict(range=[0, 20]))

fig4 = px.bar(
    df4_grouped,
    x = 'Origin Port',
    y = 'Number of Orders',
    title = 'Number of Orders per Origin Port',
    labels = {'Number of Orders': 'Orders'},
    text='Number of Orders')

fig5 = go.Figure(go.Indicator(
    mode='gauge+number',
    value=df5_ok_percentage,
    title={'text':'Percentage of On-Time or Ahead Deliveries'},
    gauge={
        'axis': {'range': [0, 100],'tickvals': [0, 20, 40, 60, 80, 90, 95, 100], 'tickangle':0},
        'bar': {'color': "#636EFA"},
        'steps': [
            {'range': [0, 90], 'color': "lightcoral"},
            {'range': [90,95], 'color': 'khaki'},
            {'range': [95, 100], 'color': "lightgreen"}]}))

### Layout ###

app.layout = html.Div([
    html.Div([
        html.Img(
            src='https://github.com/automat9/Business-Analytics/blob/master/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/visuals/Logistics%20Company%20Logo.png?raw=true',
            style={'height': '130px', 'width': '130px'}),
        
        html.H1('Logistics Insights Dashboard', style={'font-size': '60px', 'margin-left': '20px'})],
        style={'display': 'flex', 'align-items': 'center', 'border-bottom': '2px solid #ddd'}),

    html.Div([
        html.H3('Carrier Performance',style={'font-size': '24px', 'text-align': 'center', 'margin-bottom': '10px'}),
        html.Div([
            dcc.Graph(figure=fig1, style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
            dcc.Graph(figure=fig2, style={'width': '50%', 'display': 'inline-block', 'padding': '10px'})], 
            style={'display': 'flex'})], 
        style={'border-bottom': '2px solid #ddd'}),

    html.Div([
        html.H3('Operational Insights', style={'font-size': '24px', 'text-align': 'center', 'margin-bottom': '10px'}),
        html.Div([
            dcc.Graph(figure=fig3, style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
            dcc.Graph(figure=fig4, style={'width': '50%', 'display': 'inline-block', 'padding': '10px'})],
            style={'display': 'flex'})], 
        style={'border-bottom': '2px solid #ddd'}),

    html.Div([html.H3('Timely Deliveries', style={'font-size': '24px', 'text-align': 'center', 'margin-bottom': '10px'}),
        dcc.Graph(figure=fig5, style={'width': '100%'})])])

### Launch the Dashboard ###

if __name__ == '__main__':
    app.run(debug=True, port=1234)

# 5. Decision Making Process
The main aim of the project is to create a dashboard that can effectively support the managing director of a 3PL company in making informed decisions by equipping him with simple and intuitive visuals. The key steps taken during the development of this dashboard have been highlighted and discussed in the reflective blog.

### Key Steps
The process began by exploring various databases containing supply chain datasets of both fine and poor quality. The goal here was to find a dataset that addresses the requirements specified in the [First Blog Post](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=44865), and allows for the creation of a compelling scenario. The xml dataset chosen for this project contained numerous tabs and columns, which means that it closely resembles the complexity and structure of a typical supply chain company dataset. Feeding ChatGPT with the dataset instantly revealed what visuals can be extracted from it ([Second Blog Post](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=45337), Section 1). This step was only performed due to the dataset being publicly available. Using Large Language Models (LLMs) like ChatGPT for such tasks comes with the risk of data breaches, as these models collect user input for model training (Sankaran, 2024). Five visuals were selected and explored further by prompting ChatGPT to generate a code to display them ([Link](https://github.com/automat9/Business-Analytics/blob/d3229086e377cf3884c14a94adc6cc45dffa6df8/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/code/Ideas%20for%20dashboard(AI-generated).ipynb)). Exploring the visuals encouraged the designer to create a story for the presentation ([Second Blog Post](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=45337), Section 2). Adding a story when designing and presenting a dashboard allows presenters to better connect with their audience and improve engagement, making their presentation more compelling and engaging (Nussbaumer Knaflic, 2015, p. 169). Next, it was decided to finally clean and simplify the dataset by extracting relevant data and copying it onto a separate xml file (Second Blog Post, Section 3). This helped to ensure a more focused and efficient approach to analysis, as all the unnecessary data was eliminated, making it easier to develop an effective dashboard.

### Dashboard Design
Having studied the AI-generated visuals, it was decided that they presented relevant data and could be used as a framework for the final visuals. Given that the visuals focus on day-to-day performance and show metrics that can be tracked in real time, it was decided to create an operational dashboard (Few, 2006). The visuals were put together in a vertical layout, because on the sketches it looked cleaner than a horizontal layout, as discussed in the ([Third Blog Post](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=45571), Section 3). Furthermore, the vertical layout facilitated easier and more effective categorisation of the visuals into carrier performance, operational insights and timely deliveries, thus enhancing the storytelling. In the ‘vertical layout’ sketch, the dashboard has the company logo in top left corner, which serves as a branding element. Although the logo was AI-generated to suit the style of the dashboard, in a real scenario, the dashboard style would be designed to match the company logo, ensuring consistency with the company’s identity and reinforcing brand recognition (Martins, Martins, & Brandão, 2022, p. 362).

Colour coding was applied to the carriers in the ‘carrier performance’ visuals, as this allows for an easier identification, showing a clear link between both visuals and their contents ([Third Blog Post](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=45571), Section 3). In the graph that shows which products are most likely to be dispatched late, colour coding was also applied to emphasise the most problematic products, highlighting the story and drawing the managing director’s attention to the product requiring immediate action (Yi, 2019).

The values in each visual were clearly displayed above/within/below the elements. Providing specific values ensured easier reading and reduced cognitive load by eliminating the need for the managing director to cross-reference with axes or estimate sizes himself (DataVisualist, 2023).
The last figure, which shows the percentage of on-time or ahead deliveries, was also configured to tell a story. The new colour scheme is divided into red – below expected, yellow – approaching target, and green – on track. The value intervals become narrower as they as they approach the higher end. This reflects the industry standard where a good on-time delivery (OTD) rate is 95% or higher (Metrichq, 2023). This visual is bigger than the rest, and shows a positive statistic (97.9%), which helps the logistics analyst to conclude on a positive note, boost morale and motivate action (Nussbaumer Knaflic, 2015, p. 174).

One important aspect of the visuals is their interactivity, which allows the presenter to reveal more detailed data by hovering over them. This is especially useful in case the managing director wishes to dig deeper and, for example, know the specific number of orders handled by each carrier rather than just the percentage value. This enhances the decision-making process because the director has access to both high-level insights and granular data when required (Zhang, 2024).


# 6. Review of analytics methods applied
To meet the dashboard’s aim and ensure accessibility, intuitiveness and smooth storytelling, a number of analytics methods were applied.

The first section, ‘ Carrier Performance’, begins with a pie chart visualising the proportions of orders being handled by each of the three carriers. Although some experts argue that pie charts are less accurate than other types of graphs due to their reliance on area rather than length—which is usually what the eye focuses on first (Hill, 2024, p. 2) —the chart mitigates this limitation by including percentages within each element, reducing the cognitive load for the managing director. Furthermore, the presenter can hover over each element to display specific order counts, providing further insights and enhancing clarity.

Since most of the information presented was categorical, bar charts were used in three of the five visuals, as they effectively use height to compare differences between elements. Two key considerations were taken into account when designing each bar chart: ensuring the number of bars allowed for easy readability while containing sufficient information, and starting each bar chart at 0 to preserve accurate representation of the data. For example, the ‘Products by Most Late Dispatch’ visual shows the top 20 products (the dataset consists of hundreds of unique products), as this is enough to highlight that the majority of products are in the ‘green’ range, but also that some products are performing poorly (yellow and red). Putting the baseline at zero is critical in bar charts as they utilise height as the method of encoding; therefore, a zero baseline ensures accuracy and prevents misleading viewers by maintaining proportionality between the bars (Cairo, 2019, p. 61).

Lastly, the gauge diagram at the end was used to highlight the ‘sweet spot’ of on-time-delivery above 95%, which is a key performance indicator in supply chain, thus concluding the presentation on a positive note. While gauge diagrams are often recommended for scenarios where the sweet spot can be found in the middle of a range, they can be an effective way to conclude a quantitative, narrative-heavy presentation, as they provide simplicity and a visually impactful summary that leaves a good impression on the viewer (Evergreen, n.d.).

# 7. Review of selected tools and potential alternatives
The two main libraries that were used in the development of the dashboard are Dash and Plotly. 

In the bigger picture, the structure of Plotly and Dash is simple; however, challenges arose in the details, as the figures had to be constructed using complex syntaxes. The use of AI helped to simplify the process by explaining each syntax; however, some changes had to be made manually to ensure the dashboard’s intuitiveness and clarity. This was achieved through trial and error and by consulting with ChatGPT. The second challenge arose during the layout design phase, as each figure did not initially scale properly when the window was resized. This issue was also resolved through the use of AI. The final challenge was to ensure the dashboard could open on any computer without generating bugs. To address this, all important assets, such as the company logo and dataset, were uploaded to GitHub, as this provides an accessible repository for the required files. Additionally, to prevent any potential errors, a list of required libraries and installation instructions was included at the top of the dashboard code.

The dashboard also could have been developed using other tools, such as Tableau, Power BI or Bokeh.

One of Tableau’s greatest strengths is its intuitive, code-free interface combined with advanced visualisation capabilities. For this reason, it is possible that a similar dashboard could have been developed entirely in Tableau with significantly less reliance on AI support. However, unlike Dash and Plotly, Tableau is a relatively expensive tool that also has some visual customisation and data preparation limitations (Roberts, 2023). Given how much preparation each figure required, it is debatable whether Tableau would have been the most optimal choice. 
Power BI on the other hand, is a more cost-effective option that offers superior integration with Microsoft products, such as Excel, and performs significantly better in data preparation. It also features a simple and user-friendly interface; however, its ability to handle complex analytics is comparatively limited (Edmond & Crabtree, 2024). Given that the dashboard in question was not overly complex, Power BI would likely have been an adequate alternative to Dash and Plotly. 

Finally, Bokeh could have been used as an alternative to Plotly, as it is also designed to create interactive visualisations. One of its strengths is its data handling capabilities, particularly through the use of ‘ColumnDataSource’, which allows multiple graphs to share the same dataset. However, this feature also requires data to be in a specific format with equal-length columns (Iacomi, 2020). This would have been a significant limitation in the development of the dashboard in question, as it relies on columns of varying lengths.

# 8. Review of chosen dataset
The dataset that was used in this assignment was found on Kaggle.com and was uploaded by one of the users. While it remains unclear whether the dataset is anonymised data from a real company or a fictional dataset created for practice purposes, this is likely irrelevant, as the dataset is not intended to represent a factual state of affairs and thus reliability and accuracy are not a primary concern.

This dataset was chosen due to the fact that it contains many spreadsheets with a wide range of data, such as carriers, orders, storage costs, capacities and many more. This allowed to save time and ensure that all the data connects well with each other, preventing unexpected contradictions, inconsistencies, and other potential issues.

Using ChatGPT, a number of KPIs, inefficiencies and metrics were identified and taken inspiration of [(Link)](https://github.com/automat9/Business-Analytics/blob/d3229086e377cf3884c14a94adc6cc45dffa6df8/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/code/Ideas%20for%20dashboard(AI-generated).ipynb). As mentioned in the [First Blog Post](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=44865), some data cleaning needed to be carried out, as it was difficult to navigate through the original dataset due to the sheer number of data and confusing structure. A simpler version of the dataset was created and uploaded onto GitHub [(Link)](https://github.com/automat9/Business-Analytics/tree/d3229086e377cf3884c14a94adc6cc45dffa6df8/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/datasets). The simpler version only included the data that was needed to create the five visuals chosen from the list of potential visuals provided by ChatGPT.

As highlighted throughout the blog, the raw data served as the foundation upon which a narrative was constructed, providing structure and context for the analysis. The presentation starts with a clearly defined beginning (carrier performance), middle (general operational insights), and end (on-time-delivery). This approach is supported by research, which emphasises the need to create a narrative in data visualisation (Segel & Heer, 2010).

One key decision made was to not edit or encode any of the values, such as renaming carrier names from ‘V55555_53’ to something more accessible, like DHL. This was primarily to avoid confusion for those who might be interested in the original, unedited dataset. Looking back, it might have been beneficial to encode confusing names, especially the carrier names, and include a full reference list in one of the blog entries, showing the original and encoded names in relation to the dataset. This would have made the dashboard more visually appealing and accessible to the average viewer. However, assuming that the managing director is familiar with these values, the decision to leave them as unchanged can be considered appropriate for the purposes of this assignment.


# 9. Code Discussion
The following two visuals were chosen to illustrate how the final code evolved compared to the [initial AI-generated versions](https://github.com/automat9/Business-Analytics/blob/d3229086e377cf3884c14a94adc6cc45dffa6df8/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/code/Ideas%20for%20dashboard(AI-generated).ipynb). All changes made to the original versions are explicitly highlighted and discussed within the respective code sections.

The role of the gauge diagram was more narrative than analytical as explained in the dashboard design section. Its role was critical in the storytelling, as it provides a positive and meaningful conclusion to the presentation. The data required to create this diagram was very simple and did not need any complex adjustments, which was an additional argument for including this diagram in the dashboard given the project’s scope. Changing the tick values and range colours was very intuitive due to the simple syntax, and the task was achieved without any support or guidance.

The inclusion of the second visual ‘Products by Most Late Dispatch’, is critical from a 3PL perspective, as it represents a serious metric that directly impacts client satisfaction. In the worst-case scenario, repeated delays could prompt clients to switch 3PL providers. Furthermore, in many 3PL companies, the clients with the highest volumes of orders are often the largest, meaning their departure could have serious repercussions for the entire business, such as workforce downsizing. For that reason, once ChatGPT highlighted the possibility of creating this graph, it was quickly decided to include it in the final draft. Significant changes needed to be make to the initial AI-generated graph, as it was difficult to read. Fortunately, through the use of a lambda function, a simple two-line code was enough to apply colours to all the bars depending on their respective late dispatch counts. The individual values were also coded to appear above each bar, as this made the graph easier to read.


In [9]:
# AI-GENERATED CODE
# Ship on-time/ahead of schedule (gauge chart)

import dash
from dash import dcc, html
import pandas as pd
import plotly.graph_objects as go

# Load the dataset
file_path = "https://github.com/automat9/Business-Analytics/raw/master/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/datasets/data.xlsx"
data = pd.read_excel(file_path)

# Calculate total deliveries and on-time/ahead deliveries
total_deliveries = len(data)
on_time_or_ahead_deliveries = len(data[(data['Ship Late Day count'] <= 0)])

# Calculate the percentage of on-time or ahead deliveries
on_time_or_ahead_percentage = (on_time_or_ahead_deliveries / total_deliveries) * 100

# Create the gauge chart
fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=on_time_or_ahead_percentage,
    title={'text': "Percentage of On-Time or Ahead Deliveries"},
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': "blue"},
        'steps': [
            {'range': [0, 50], 'color': "lightcoral"},
            {'range': [50, 100], 'color': "lightgreen"}
        ]
    }
))

# Initialize Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("On-Time or Ahead Delivery Performance"),
    dcc.Graph(id='gauge-chart', figure=fig)
])

# Run app
if __name__ == '__main__':
    app.run_server(debug=True, port=8000)

In [11]:
# Improved and condensed Ship on-time/ahead of schedule (gauge chart)
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

##########################################################################################
# Condensed code to ensure simplicity and better readibility
data = pd.read_excel("https://raw.githubusercontent.com/automat9/Business-Analytics/4345e44423e83e68e5e92815c37036670c9acad8/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/datasets/data_cleaned.xlsx",sheet_name=None)

df5 = data['ontime_ahead_gauge'] 

total_deliveries = len(df5)
df5_ok_deliveries = len(df5[df5['Ship Late Day count'] <= 0])
df5_ok_percentage = (df5_ok_deliveries / total_deliveries) * 100
##########################################################################################

app = Dash(__name__)

fig5 = go.Figure(go.Indicator(
    mode='gauge+number',
    value=df5_ok_percentage,
    title={'text':'Percentage of On-Time or Ahead Deliveries'},
    gauge={
##########################################################################################
# Changed tick values to provide more details as we get closer to the 95% OTD
# Added a 'khaki' range between 90% and 95% to represent a transition zone between 'good' and 'bad' OTD percentages
# Changed the colour of the blue bar to make it less saturated and aggressive
        'axis': {'range': [0, 100],'tickvals': [0, 20, 40, 60, 80, 90, 95, 100], 'tickangle':0},
        'bar': {'color': "#636EFA"},
        'steps': [
            {'range': [0, 90], 'color': "lightcoral"},
            {'range': [90,95], 'color': 'khaki'},
            {'range': [95, 100], 'color': "lightgreen"}]}))
##########################################################################################

# AI-GENERATED CODE TO DISPLAY THE INDIVIDUAL GRAPH
app.layout = html.Div([
    html.H1('Timely Delivery Insights', style={'font-size': '40px', 'text-align': 'center'}),
    dcc.Graph(figure=fig5, style={'width': '80%', 'margin': 'auto'})])


if __name__ == '__main__':
    app.run(debug=True, port=8001)

In [13]:
# AI-GENERATED CODE
# Top 20 products by late dispatch (bar chart)


# Import libraries
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

# File location
url = "https://github.com/automat9/Business-Analytics/raw/master/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/datasets/data.xlsx"
# Load and prepare data
top_10_products_df = (
    pd.read_excel(url, sheet_name='OrderList')
    .query('`Ship Late Day count` > 0')
    .assign(**{'Product ID': lambda df: df['Product ID'].astype(str)})
    ['Product ID']
    .value_counts()
    .nlargest(20)
    .reset_index(name='Late Order Count')
    .rename(columns={'index': 'Product ID'})
)

# Initialise the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Graph(
        id='late-orders-bar-chart',
        figure=px.bar(
            top_10_products_df,
            x='Product ID',
            y='Late Order Count',
            title='Products by Most Late Orders',
            labels={'Product ID': 'Product ID', 'Late Order Count': 'Number of Late Orders'}
        ).update_traces(marker_color='steelblue')
    )
])

# Run the app
app.run_server(mode='inline', port=8002, dev_tools_ui=True, dev_tools_props_check=True)

In [15]:
# Improved and condensed Top 20 products by late dispatch (bar chart)
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

data = pd.read_excel("https://raw.githubusercontent.com/automat9/Business-Analytics/4345e44423e83e68e5e92815c37036670c9acad8/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/datasets/data_cleaned.xlsx", sheet_name=None)

df3 = data['top 20_late_bar']

df3_ordered = (
    df3.query('`Ship Late Day count` > 0')
    ['Product ID']
    .astype(str)
    .value_counts()
    .nlargest(20)
    .reset_index(name='Late Order Count')
    .rename(columns={'index': 'Product ID'}))
##########################################################################################
# Introducing a code that assigns a colour to each bar depending on its value
df3_ordered['colour'] = df3_ordered['Late Order Count'].apply(
    lambda x: 'lightgreen' if x < 10 else 'khaki' if 10 <= x <= 13 else 'lightcoral')
##########################################################################################

app = Dash(__name__)


fig3 = px.bar(
    df3_ordered,
    x='Product ID',
    y='Late Order Count',
    labels={'Late Order Count': 'Late Dispatch'},
    title='Products by Most Late Dispatch',
##########################################################################################
# the text= code adds values above the bars to improve readibility
# the dict(range=[0,20]) code ensures that the graph is large enough to display all value
# (note how in the original AI-generated graph, there is not enough space to fit the '17' above the first bar)
    text='Late Order Count')
fig3.update_traces(marker=dict(color=df3_ordered['colour']),texttemplate='%{text}', textposition='outside')
fig3.update_layout(yaxis=dict(range=[0, 20]))


# AI-GENERATED CODE TO DISPLAY THE INDIVIDUAL GRAPH
app.layout = html.Div([
    html.H1('Top 20 Products by Late Dispatch', style={'font-size': '40px', 'text-align': 'center'}),
    dcc.Graph(figure=fig3, style={'width': '80%', 'margin': 'auto'})])


if __name__ == '__main__':
    app.run(debug=True, port=8003)

# 10. Personal Reflection
This section will use the Gibbs’ Reflective Cycle as it provides a clear structure for personal reflection (Galli & New, 2022).

The approach taken in the development of the dashboard was backwards in the sense that the dashboard was created first, and then the objectives, scenario, and relevant literature were reviewed after the development of the dashboard. Given the fact that the researcher had to find his own dataset and create a dashboard that would fit its content, he felt that this was the best option for creating a comprehensive and consistent setting. Furthermore, during the development of the previous assignment, which was a data visualisation critique, the researcher learned many aspects of what can be considered ‘good visuals’ and what choices were not optimal, which helped to guide his decisions in the development of the dashboard. This is why this time the literature review was conducted last. The researcher acknowledges that research should have been carried out before starting to create the dashboard, as dashboards and visuals are not the same thing. However, although the steps in this assignment were not chronological, in a real-world setting where the researcher is familiar with what kind of data is stored by his company and is aware of what type of visuals can be extracted from it, this assignment helped significantly in understanding the necessary considerations that need to be made. As such, in a future scenario where a dataset is provided and the researcher is familiar with good visualisation practices, the dashboard development process will start by properly defining aims and objectives, followed by dashboard development, and finally creating a compelling story to address each graph. This will ensure that each graph not only presents data effectively but also contributes to a narrative aligned with the defined objectives.

The extensive use of AI in the development of the dashboard was also something that, in a real-world setting, would not be possible due to data protection policies. The ideas for graphs were generated after feeding ChatGPT with the dataset and prompting it to analyse it. This helped save time and better understand the dataset, which eased the creation of the scenario and objectives, making the researcher feel more confident about the project. The use of AI in the development of the dashboard was initially seen as a drawback, as the code was not written by a human, and it did not feel like an accomplishment for the researcher. However, through careful and iterative prompting, the code became easier to understand as ChatGPT explained every section (processing data, creating graphs, and layout) in great detail. As such, future dashboards will require less reliance on AI, ensuring compliance with company data protection policies.


# 11. Conclusion
This assignment has helped the researcher to understand the dashboard development process and prepare him for real-world situations where a dashboard will need to be created. The dashboard itself presents important metrics in an intuitive and clear way. Combined with the carefully developed scenario, this makes it an effective tool for educating and influencing senior management, ensuring swift and data-driven decision-making.


# 12. References
1)	Cairo, A. (2019). How charts lie: Getting smarter about visual information. W. W. Norton Trade.
  
2)	DataVisualist. (2023, September 25). Optimizing information dashboard design: The influences of cognitive load theory. DataVisualist. https://www.datavisualist.com/post/optimizing-information-dashboard-design-the-influences-of-cognitive-load-theory
  
3)	Edmond, S., & Crabtree, M. (2024, November 25). Power BI vs Tableau: Which one should you choose? DataCamp. Retrieved December 10, 2024, from https://www.datacamp.com/blog/power-bi-vs-tableau-which-one-should-you-choose
  
4)	Evergreen, S. (n.d.). Gauge diagrams: Are they worth it? Stephanie Evergreen. Retrieved December 9, 2024, from https://stephanieevergreen.com/gauge-diagram/
 
5)	Few, S. (2006). Information dashboard design: The effective visual communication of data. O’Reilly Media.
   
6)	Galli, F., & New, K. J. (2022). Gibbs’ cycle review - Emotions as a part of the cycle. Revista de Educación, Motricidad e Investigación, 19, 92–101. https://doi.org/10.33776/remo.vi19.7224
   
7)	Hill, A. (2024). Are pie charts evil? An assessment of the value of pie and donut charts in business and media. Information Visualization, 1–21. https://doi.org/10.1177/14738716241259432
   
8)	Iacomi, P. (2020, June 7). Plotly vs Bokeh: Interactive Python visualisation libraries compared. Retrieved December 10, 2024, from https://pauliacomi.com/2020/06/07/plotly-v-bokeh.html

9)	Martins, N., Martins, S., & Brandão, D. (2022). Design principles in the development of dashboards for business management. In D. Raposo, J. Neves, & J. Silva (Eds.), Perspectives on design II (Vol. 16, pp. 353–365). Springer. https://doi.org/10.1007/978-3-030-79879-6_26
   
10)	Metrichq. (2023, September 14). On-time delivery: What it is and how to improve it. Metrichq. https://www.metrichq.org/supply-chain/on-time-delivery
   
11)	Nussbaumer Knaflic, C. (2015). Storytelling with data: A data visualization guide for business professionals (1st ed.). Wiley.
   
12)	Roberts, S. (2023, October 18). Advantages and disadvantages of Tableau: Detailed explanation. The Knowledge Academy. Retrieved December 9, 2024, from https://www.theknowledgeacademy.com/blog/advantages-and-disadvantages-of-tableau/
   
13)	Sankaran, V. (2024, January 9). OpenAI says it is ‘impossible’ to train AI without using copyrighted works for free. The Independent. https://www.independent.co.uk/tech/openai-chatgpt-copyrighted-work-use-b2475386.html
   
14)	Segel, E., & Heer, J. (2010). Narrative visualization: Telling stories with data. IEEE Transactions on Visualization and Computer Graphics, 16(6), 1139–1148. https://doi.org/10.1109/TVCG.2010.179
   
15)	Yi, M. (2019, October 24). How to choose colors for your data visualizations. Nightingale. https://nightingaledvs.com/how-to-choose-colors-for-your-data-visualizations/
   
16)	Zhang, Q. (2024). The impact of interactive data visualization on decision-making in business intelligence. Advances in Economics, Management and Political Sciences, 87, 166–171. https://doi.org/10.54254/2754-1169/87/20241056
